In [107]:
ACCESSORIES_TO_COLUMN_NAME = {
    'Airconditioning: Automatic': 'airconditioning',
    'Parking sensors': 'parking_sensors',
    'Satellite navigator': 'satellite_navigator',
    'Cruise control: Adaptive': 'cruise_control_adaptive',
    'Parking camera: Simple camera': 'parking_camera_simple_camera',
    'Seat heaters': 'seat_heaters',
    'Driving assistant': 'driving_assistant',
    'Alloy wheels': 'alloy_wheels',
    'Electric mirrors': 'electric_mirrors',
    'Lane departure warning system': 'lane_departure_warning_system',
    'Electrically operated tailgate': 'electrically_operated_tailgate',
    'Leather upholstery': 'leather_upholstery',
    'Emergency brake assist': 'emergency_brake_assist',
    'Collision avoidance system': 'collision_avoidance_system',
    'Cruise control: Traditional': 'cruise_control_traditional',
    'Parking Assistant': 'parking_assistant',
    'Tow bar': 'tow_bar',
    'Electric seats: With memory': 'electric_seats_with_memory',
    'Heated steering wheel': 'heated_steering_wheel',
    'Sunroof': 'sunroof',
    'Sun hatch: With panorama': 'sun_hatch_with_panorama',
    'Adaptive headlights': 'adaptive_headlights',
    'Sport seats': 'sport_seats',
    'Fuel / battery powered heater': 'fuel_battery_powered_heater',
    'Parking camera: 360-degree camera': 'parking_camera_360-degree_camera',
    'Battery preheating': 'battery_preheating',
    'Electric seats: Without memory': 'electric_seats_without_memory',
    'Air suspension': 'air_suspension',
    'Curve lights': 'curve_lights',
    'Head-Up display': 'head_up_display',
    'Sport base': 'sport_base,'
}

DRIVE_TYPE = {'Four wheel': '4wd', 'Front wheel': 'fwd', 'Rear wheel': 'rwd', 'Not available': None}

fields_to_pick = [
    'make',
    'model',
    'modelTypeName',
    'color',
    'driveType',
    'price',
    'totalOwners',
    'kilometers',
    'seats',
    'power',
    'batteryCapacity',
    'electricRange',
]

columns = [
    *fields_to_pick,
    'isSuv',
    'metallicColor',
    *ACCESSORIES_TO_COLUMN_NAME.values(),
]

In [142]:
import re

zero_times_s = 'Pro Performance 1ST 150 kW'
one_times_s = 'Pro Performance 1ST 150 kW, akku 77 kWh'
two_times_s = 'Pro Performance 1ST 150 kW, akku 77 kWh Pro Performance 1ST 150 kW, akku 88 kWh'
one_time_no_space = 'Pro Performance 1ST 150 kW, akku 77kWh Pro Performance 1ST 150 kW'



battery_regex = r'\d{2,3}\s*kWh'

def parse_battery_capacity_from_free_text(free_text):
    matches = re.findall(battery_regex, free_text, re.IGNORECASE)
    if not len(matches):
        return 0
    return sorted([int(s.lower().replace('kwh', '').split()[0]) for s in matches], reverse=True)[0]

print(parse_battery_capacity_from_free_text(zero_times_s))
print(parse_battery_capacity_from_free_text(one_times_s))
print(parse_battery_capacity_from_free_text(two_times_s))
print(parse_battery_capacity_from_free_text(one_time_no_space))

0
77
88
77


In [202]:
import os
import json
import pandas as pd

QUERY_RESULTS_PATH = './query_results'



def open_nettiauto_file(file_name):
    print(file_name)
    with open(f'{QUERY_RESULTS_PATH}/{file_name}', 'r') as f:
        data = json.load(f)
    return data

def map_nettiauto_entry_to_row(item):
    try:
        acc = item['accessories'] if item['accessories'] else []
        acc_set = set([ ACCESSORIES_TO_COLUMN_NAME[row['en']] for row in acc if row['en'] in ACCESSORIES_TO_COLUMN_NAME])


        result_without_accessories = {key: item[key] for key in fields_to_pick if key in item}
        result_without_accessories['isSuv'] = item['bodyType']['id'] == 5
        result_without_accessories['make'] = result_without_accessories['make']['name'].lower()
        result_without_accessories['model'] = result_without_accessories['model']['name'].lower()
        result_without_accessories['color'] = result_without_accessories['color']['en'].lower()
        result_without_accessories['metallicColor'] = item['colorType']['en'].lower() == 'metallic'
        result_without_accessories['driveType'] = DRIVE_TYPE[result_without_accessories['driveType']['en']]
        
        # It's ugly but does its job. I was able to get the missing batteryCapacity from 2109 to 1219
        if not result_without_accessories['batteryCapacity'] and 'modelTypeName' in item and item['modelTypeName']:
            result_without_accessories['batteryCapacity'] = parse_battery_capacity_from_free_text(item['modelTypeName'])
        if not result_without_accessories['batteryCapacity'] and 'description' in item and item['description']:
            result_without_accessories['batteryCapacity'] = parse_battery_capacity_from_free_text(item['description'])

        return {
            **result_without_accessories,
            **{key: key in acc_set for key in ACCESSORIES_TO_COLUMN_NAME.values()},
        }
    except Exception as e:
        print(item['id'])
        raise e


dfs = []

for file_name in sorted(os.listdir(QUERY_RESULTS_PATH)):
    results = map(map_nettiauto_entry_to_row, open_nettiauto_file(file_name))
    dfs.append(pd.DataFrame(data=results, columns=columns))

    
df = pd.concat(dfs)
df.head()


2021-01_response_1696071482757.json
2021-02_response_1696071737045.json
2021-03_response_1696072016857.json
2021-04_response_1696072267024.json
2021-05_response_1696072588298.json
2021-06_response_1696072850927.json
2021-07_response_1696073239785.json
2021-08_response_1696073507847.json
2021-09_response_1696073655890.json
2022-01_response_1696074468991.json
2022-02_response_1696074615821.json
2022-03_response_1696074839641.json
2022-04_response_1696075021170.json
2022-05_response_1696075239764.json
2022-07_response_1696075876798.json
2022-07_response_1696079148457.json
2022-08_response_1696076147323.json
2022-09_response_1696076364847.json
2022-10_response_1696076596294.json
2022-11_response_1696076798891.json
2022-12_response_1696076991892.json
2022-13_response_1696077143377.json
2022-14_response_1696077274819.json
2022_06_response_1696075611367.json
2023-01_response_1696078129488.json
2023-02_response_1696078250632.json
2023-03_response_1696078413236.json
2023-04_response_16960785579

,make,model,modelTypeName,color,driveType,price,totalOwners,kilometers,seats,power,...,adaptive_headlights,sport_seats,fuel_battery_powered_heater,parking_camera_360-degree_camera,battery_preheating,electric_seats_without_memory,air_suspension,curve_lights,head_up_display,"sport_base,"
0,porsche,taycan,Taycan sähköauto Heti toimitukseen!! rahoitus ...,black,rwd,101880.0,1.0,2100.0,5.0,300.0,...,False,True,False,False,False,True,False,False,False,True
1,opel,zafira-e,"Life L Comfort 136 automaatti 50 **Täyssähkö, ...",other,fwd,47900.0,NaN,2500.0,NaN,100.0,...,False,False,False,False,False,False,False,False,False,False
2,mercedes-benz,eqs,"580 4Matic ** Muistipenkit, Burmester, nelipyö...",gray,4wd,125900.0,NaN,30000.0,5.0,385.0,...,True,True,True,True,True,False,True,False,False,False
3,nissan,leaf,e+ N-Connecta MY21 62 kWh *Isoakkunen Huippuva...,black,fwd,35800.0,NaN,10000.0,5.0,160.0,...,False,False,False,False,False,False,False,False,False,False
4,citroen,e-c4,"Full Electric 136 Shine 50 kWh, KORKOTARJOUS 3...",blue,fwd,33800.0,NaN,17000.0,5.0,100.0,...,False,False,False,False,False,False,False,False,False,False


In [204]:
toyotas_no_battery = no_battery_df[no_battery_df['make'] == 'toyota']['model']
toyotas_no_battery.value_counts()

model
bz4x    34
Name: count, dtype: int64

In [205]:
mercedes = df[(df['make'] == 'mercedes-benz') & (df['model'] == 'eqe')]
print(mercedes['batteryCapacity'].value_counts())
mercedes['modelTypeName']

batteryCapacity
0.0      78
90.0     25
89.0     24
90.6     11
100.0     7
91.0      3
98.0      2
88.1      1
56.0      1
Name: count, dtype: int64


16    350+ #Suomi-auto  #Panoraamalasikatto #Premium...
17    Mercedes-AMG EQE 43 4MATIC #Suomi-auto #Premiu...
18    350+ #AMG Line #Suomi-auto #Premium #Digital L...
19                      350+ *** Korko alk. 1,95% !!! *
43    350+ #Suomi-auto #Premium #AMG-ulko-/sisä #AMG...
                            ...                        
89    500 4MATIC SUV ** Esittelyauto nopeaan toimitu...
92                                           350 4MATIC
97    350 4matic ** Esittelyauto nopeaan toimituksee...
7                                        350 4MATIC SUV
14    500 4MATIC SUV #Heti toimitus #AMG-Line #Nelip...
Name: modelTypeName, Length: 152, dtype: object

In [206]:
mask = (df['make'] == 'mercedes-benz') & (df['model'] == 'eqe') & (df['modelTypeName'].str.contains(r'(350|300)\s', regex=True))
df.loc[mask, 'batteryCapacity'] = 89.0
mask = (df['make'] == 'mercedes-benz') & (df['model'] == 'eqe') & (df['modelTypeName'].str.contains(r'(350\+|500|43)\s', regex=True))
df.loc[mask, 'batteryCapacity'] = 90.6
df[(df['batteryCapacity'] == 0) & (df['make'] == 'mercedes-benz') & (df['model'] == 'eqe')]['modelTypeName']


/var/folders/3r/zp_fcf693r3g5yq1xc11l1j00000gn/T/ipykernel_37952/2683371586.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask = (df['make'] == 'mercedes-benz') & (df['model'] == 'eqe') & (df['modelTypeName'].str.contains(r'(350|300)\s', regex=True))
/var/folders/3r/zp_fcf693r3g5yq1xc11l1j00000gn/T/ipykernel_37952/2683371586.py:3: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask = (df['make'] == 'mercedes-benz') & (df['model'] == 'eqe') & (df['modelTypeName'].str.contains(r'(350\+|500|43)\s', regex=True))


49    2xAMG-Line, Distronic+, Panorama, Muistipakett...
99                                                 None
5                                                  None
51    AMG-Line SE-edition, hinta sis.alv 24%, vetoko...
51                                                 None
43                                                 None
52    Advanced Sound System, Nahkaverhoilu, LED High...
23                                                 None
69                                                 None
11                                                 None
61                                                 None
9                                                   300
Name: modelTypeName, dtype: object

In [207]:
mask = (df['make'] == 'mercedes-benz') & (df['model'] == 'eqs') & (df['modelTypeName'].str.contains(r'(450|450\+|580|53)\s', regex=True))
mercedes = df[mask]
print(mercedes['batteryCapacity'].value_counts())
mercedes['modelTypeName']

batteryCapacity
0.0      34
108.0    19
108.4     6
107.8     4
120.0     3
112.0     2
107.0     1
Name: count, dtype: int64


/var/folders/3r/zp_fcf693r3g5yq1xc11l1j00000gn/T/ipykernel_37952/774634179.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask = (df['make'] == 'mercedes-benz') & (df['model'] == 'eqs') & (df['modelTypeName'].str.contains(r'(450|450\+|580|53)\s', regex=True))


2     580 4Matic ** Muistipenkit, Burmester, nelipyö...
19    580 4Matic Electric Art / Night / designo / Pr...
28    580 4Matic - ** Nelipyöräohjaus / Koukku / Bur...
38    580 4Matic *AMG, Premium Plus, Panorama, Night...
60    450+ **AMG Line, Panoraama, Burmester, Koukku,...
                            ...                        
0                     580 4MATIC *NOPEAAN TOIMITUKSEEN*
9     53 AMG 4MATIC+ **762HP, HUIPPUVARUSTEET & KORK...
75                                       450 4MATIC SUV
0     450 4MATIC SUV ** Esittelyauto nopeaan toimitu...
12    580 4Matic AMG Burmester/HUD/Nahkaverhoilu/Ilm...
Name: modelTypeName, Length: 69, dtype: object

In [208]:
# https://www.mercedes-benz.fi/content/dam/finland/passengercars/NEW-WLTP-pricelists/EQS/EQS_25042023_1.pdf
mask = (df['make'] == 'mercedes-benz') & (df['model'] == 'eqs') & (df['modelTypeName'].str.contains(r'(450|450\+|580|53)\s', regex=True))
df.loc[mask, 'batteryCapacity'] = 108.4
mask = (df['make'] == 'mercedes-benz') & (df['model'] == 'eqs') & (df['modelTypeName'].str.contains(r'(350)\s', regex=True))
df.loc[mask, 'batteryCapacity'] = 90.6

/var/folders/3r/zp_fcf693r3g5yq1xc11l1j00000gn/T/ipykernel_37952/985649123.py:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask = (df['make'] == 'mercedes-benz') & (df['model'] == 'eqs') & (df['modelTypeName'].str.contains(r'(450|450\+|580|53)\s', regex=True))
/var/folders/3r/zp_fcf693r3g5yq1xc11l1j00000gn/T/ipykernel_37952/985649123.py:4: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask = (df['make'] == 'mercedes-benz') & (df['model'] == 'eqs') & (df['modelTypeName'].str.contains(r'(350)\s', regex=True))


In [209]:
mask = (df['make'] == 'mercedes-benz') & (df['model'] == 'eqa') & (df['modelTypeName'].str.contains(r'\d{3}\+\s', regex=True))
mercedes = df[mask]
print(mercedes['batteryCapacity'].value_counts())
mercedes['modelTypeName']

batteryCapacity
66.0     1
100.0    1
70.0     1
0.0      1
Name: count, dtype: int64


63    250+ suuremmalla akulla Advanced 529km (WLTP) ...
15    250+ Business / Koukku / AMG Line / EQ-Navigoi...
26     250+ Business **Distronic, MBUX-inovaatiopaketti
17                                        250+ Business
Name: modelTypeName, dtype: object

In [211]:
# https://www.mercedes-benz.fi/content/dam/finland/passengercars/NEW-WLTP-pricelists/EQA/EQA_10.5.2023_1.pdf
mask = (df['make'] == 'mercedes-benz') & (df['model'] == 'eqa') & (df['modelTypeName'].str.contains(r'\d{3}\+\s', regex=True))
df.loc[mask, 'batteryCapacity'] = 70.5
mask = (df['make'] == 'mercedes-benz') & (df['model'] == 'eqa') & (df['modelTypeName'].str.contains(r'\d{3}\s', regex=True))
df.loc[mask, 'batteryCapacity'] = 66.5

In [212]:
mask = (df['make'] == 'mercedes-benz') & (df['model'] == 'eqb') & (df['modelTypeName'].str.contains(r'\d{3}\+\s', regex=True))
mercedes = df[mask]
print(mercedes['batteryCapacity'].value_counts())
mercedes['modelTypeName']

Series([], Name: count, dtype: int64)


Series([], Name: modelTypeName, dtype: object)

In [213]:
mask = (df['make'] == 'mercedes-benz') & (df['model'] == 'eqb') & (df['modelTypeName'].str.contains(r'\d{3}\+\s', regex=True))
df.loc[mask, 'batteryCapacity'] = 70.5
mask = (df['make'] == 'mercedes-benz') & (df['model'] == 'eqb') & (df['modelTypeName'].str.contains(r'\d{3}\s', regex=True))
df.loc[mask, 'batteryCapacity'] = 66.5

In [214]:
mask = (df['make'] == 'mercedes-benz') & (df['model'] == 'eqc')
df.loc[mask, 'batteryCapacity'] = 80

In [215]:
no_battery_df = df[df['batteryCapacity'] == 0]
mercedes_no_battery = no_battery_df[no_battery_df['make'] == 'mercedes-benz']['model']
mercedes_no_battery.value_counts()

model
eqe        12
eqs        10
eqa         8
eqs suv     8
eqb         7
eqv         3
vito        1
Name: count, dtype: int64

In [216]:
mask = (df['make'] == 'bmw') & (df['model'] == 'ix') & (df['modelTypeName'].str.contains(r'xDrive\s*40', regex=True))
print(df[mask][['modelTypeName', 'batteryCapacity']])
df.loc[mask, 'batteryCapacity'] = 76.6
mask = (df['make'] == 'bmw') & (df['model'] == 'ix') & -mask
df.loc[mask, 'batteryCapacity'] = 111.5

                                        modelTypeName  batteryCapacity
92  xDrive40 // BMW Premium Selection -takuu 24kk/...              0.0
20  xDrive40 Fully Charged / Adapt.cruise / HUD / ...             76.6
28  xDrive40 *SUPERVARUSTEET! 22"/LASER/HUD/PANORA...              0.0
88  xDrive40 Fully Charged, WLTP 372km, Harman/Kar...              0.0
71  xDrive40 Fully Charged 11CF *** Korko alk. 1,9...              0.0
88  xDrive40, WLTP 372km, Sähkösäätöiset Etuistuim...             71.0
15  xDrive40 Fully Charged *ESITTELYAUTO* Hinta uu...              0.0
42  xDrive40 Fully Charged // Huippuvarusteet  **B...              0.0
62        xDrive40 *** BMW Rahoitusetu 2,90% (+kulut)              0.0
63  Akku 77kWh xDrive40 Fully Charged Aut. 326hv |...             77.0
85  xDrive40, WLTP 372km, HUD, Kamera, Sähkösäätöi...             71.0
12  xDrive40 Fully Charged // Sport paketti/Ajoavu...              0.0
82  xDrive40 Fully Charged, WLTP 372km, Innovation...             71.0
83  xD

In [217]:
no_battery_df = df[df['batteryCapacity'] == 0]
bmw_no_battery = no_battery_df[no_battery_df['make'] == 'bmw']['model']
bmw_no_battery.value_counts()

model
ix1       33
i4        29
i4 m50    25
ix3       19
i7         5
i3         4
i3s        3
ix m60     3
Name: count, dtype: int64

In [218]:
no_battery_df = df[df['batteryCapacity'] == 0]
no_battery_df['make'].value_counts()
print(sum(df['batteryCapacity'] > 0) / len(df))
print(sum(df['batteryCapacity'] == 0))

0.7618595825426945
1004
